In [ ]:
###############################################################################
import aliases

In [ ]:
from everest.ptolemaic.chora import *

In [ ]:
import abc as _abc
import functools as _functools
import inspect as _inspect
import collections as _collections

from everest.abstract import *
from everest.utilities import (
    TypeMap as _TypeMap, MultiTypeMap as _MultiTypeMap
    )

from everest.ptolemaic.essence import Essence as _Essence
from everest.ptolemaic.ptolemaic import Ptolemaic as _Ptolemaic, DataPtolemaic as _DataPtolemaic

In [ ]:
# class Bounds(_DataPtolemaic):

#     FIELDS = ('lbnd', 'ubnd')


# class Sampler(_DataPtolemaic):

#     FIELDS = ('sampler',)


class Traversable(Sliceable):

#     def getitem_slice(self, caller, incisor: slice, /):
#         lbnd, ubnd, sampler = (incisor.start, incisor.stop, incisor.step)
#         bounds, sampler = Bounds(lbnd, ubnd), Sampler(sampler)
#         return caller[bounds][sampler]

#     def getitem_bounds(self, caller, incisor: Bounds, /):
#         lbnd, ubnd = incisor.lbnd, incisor.ubnd
#         meth = self.bndgetmeths[type(lbnd), type(ubnd)]
#         return meth(self, caller, lbnd, ubnd)

#     def getitem_sampler(self, caller, incisor: Sampler, /):
#         sampler = incisor.sampler
#         meth = self.samplegetmeths[type(sampler)]
#         return meth(self, caller, sampler)

    def slice_incise_bound(self, caller,
            lbnd: object, ubnd: object, _: NoneType, /
            ):
        meth = self.bndgetmeths[type(lbnd), type(ubnd)]
        return meth(self, caller, lbnd, ubnd)

    def slice_incise_sample(self, caller,
            _: NoneType, __: NoneType, sampler: NotNone, /
            ):
        meth = self.samplegetmeths[type(sampler)]
        return meth(self, caller, sampler)

    def slice_incise_boundsample(self, caller,
            lbnd: object, ubnd: object, sampler: NotNone, /
            ):
        '''Equivalent to `caller[lbnd:ubnd][::sampler]`.'''
        return (
            self.slice_incise_bound(self, lbnd, ubnd, None)
            .slice_incise_sample(caller, None, None, sampler)
            )
            

    def bound_trivial_none(self, caller, lbnd: NoneType, ubnd: NoneType):
        '''Captures the special behaviour implied by `self[:]`.'''
        return caller

    def sample_trivial_none(self, caller, sampler: NoneType):
        '''Captures the special behaviour implied by `self[::None]`.'''
        return caller

    def sample_trivial_ellipsis(self, caller, sampler: NoneType):
        '''Captures the special behaviour implied by `self[::...]`.'''
        return caller

    @classmethod
    def _yield_bndmeths(cls, /):
        for name in cls.attributes:
            for prefix in map('bound_'.__add__, cls.PREFIXES):
                if name.startswith(prefix + '_'):
                    yield name, getattr(cls, name)
                    break

    @classmethod
    def _get_bndmeths(cls, /):
        return dict(cls._yield_bndmeths())

    @classmethod
    def _yield_bndgetmeths(cls, /):
        for meth in cls.bndmeths.values():
            hint = tuple(meth.__annotations__.values())[:3]
            yield hint, meth

    @classmethod
    def _get_bndgetmeths(cls, /):
        return _MultiTypeMap(cls._yield_bndgetmeths())

    @classmethod
    def _yield_samplemeths(cls, /):
        for name in cls.attributes:
            for prefix in map('sample_'.__add__, cls.PREFIXES):
                if name.startswith(prefix + '_'):
                    yield name, getattr(cls, name)
                    break

    @classmethod
    def _get_samplemeths(cls, /):
        return dict(cls._yield_samplemeths())

    @classmethod
    def _yield_samplegetmeths(cls, /):
        return (
            (next(iter(meth.__annotations__.values())), meth)
            for meth in cls.samplemeths.values()
            )

    @classmethod
    def _get_samplegetmeths(cls, /):
        return _TypeMap(cls._yield_samplegetmeths())

    @classmethod
    def __class_init__(cls, /):
        super().__class_init__()
        cls.bndmeths = cls._get_bndmeths()
        cls.bndgetmeths = cls._get_bndgetmeths()
        cls.samplemeths = cls._get_samplemeths()
        cls.samplegetmeths = cls._get_samplegetmeths()

In [ ]:
class MyChora(Traversable, _Ptolemaic):

    __slots__ = ('lbnd', 'ubnd', 'sampler', 'rangeobj')

    def __init__(self, lbnd: int, ubnd: int, sampler: int = 1, /):
        self.lbnd, self.ubnd, self.sampler = lbnd, ubnd, sampler
        self.rangeobj = range(lbnd, ubnd, sampler)
        super().__init__()

    def retrieve_int(self, caller, incisor: int, /):
        return caller.retrieve(self.rangeobj[incisor])

    def _process_lbnd(self, lbnd: int, /):
#         if lbnd < 0:
#             return self.ubnd + self.sampler * lbnd
        return self.lbnd + self.sampler * lbnd

    def _process_ubnd(self, ubnd: int, /):
#         if ubnd < 0:
#             return self.ubnd + self.sampler * ubnd
        return self.lbnd + self.sampler * ubnd + 1

    def bound_incise_int_lower(self, caller, lbnd: int, ubnd: NoneType, /):
        lbnd = self._process_lbnd(lbnd)
        return caller.incise(type(self)(lbnd, self.ubnd, self.sampler))

    def bound_incise_int_upper(self, caller, lbnd: NoneType, ubnd: int, /):
        ubnd = self._process_ubnd(ubnd)
        return caller.incise(type(self)(self.lbnd, ubnd, self.sampler))

    def bound_incise_int_dual(self, caller, lbnd: int, ubnd: int, /):
        lbnd, ubnd = self._process_lbnd(lbnd), self._process_ubnd(ubnd)
        return caller.incise(type(self)(lbnd, ubnd, self.sampler))

    def sample_incise_int(self, caller, sampler: int, /):
        sampler = self.sampler * sampler
        return caller.incise(type(self)(self.lbnd, self.ubnd, sampler))

    def __iter__(self, /):
        return self.iter_(self)

    def iter_(self, caller, /):
        return map(caller.retrieve, range(self.lbnd, self.ubnd, self.sampler))

    def __contains__(self, arg, /):
        return arg in self.rangeobj

    def _repr(self, /):
        return ':'.join(map(repr, (self.lbnd, self.ubnd, self.sampler)))

    def get_epitaph(self, /):
        return self.taphonomy.custom_epitaph(
            '$A($a,$b,$c)',
            dict(
                A=type(self)._ptolemaic_class__,
                a=self.lbnd, b=self.ubnd, c=self.sampler,
                ),
            )

In [ ]:
mychora = MyChora(0, 100, 1)
mychora

In [ ]:
mychora[30:70][:35][::2][5:][3:15:3].rangeobj

In [ ]:
mychora.rangeobj[30:70][:35][::2][5:][3:15:3]

In [ ]:
mychora[30:50][:15][::2][5::3].rangeobj

In [ ]:
mychora.rangeobj[30:50][:15][::2][5::3]

In [ ]:
tuple(mychora[::2][::3])

In [ ]:
mychora[10:-20:2][15:]

In [ ]:
mychora[15:] is mychora

In [ ]:
mychora[15:30:2]

In [ ]:
mychora[::2]

In [ ]:
assert mychora[:] is mychora
assert mychora[...] is mychora
assert mychora[None] is mychora
assert mychora[None:None:None] is mychora
assert mychora[::] is mychora
assert mychora[::None] is mychora
assert mychora[:None:None] is mychora
assert mychora[None::None] is mychora
assert mychora[None::] is mychora

In [ ]:
from everest.utilities import caching as _caching


@Chora.decorate
class MyClass(Ptolemaic):

    __slots__ = ('dct',)

    def __init__(self, dct, /):
        self.dct = dct

    @property
    @_caching.soft_cache()
    def chora(self, /):
        return self.Chora()

    def retrieve(self, index):
        return self.dct[index]

    def __contains__(self, arg, /):
        return arg in self.dct

    def _repr(self, /):
        return self.dct

    def get_epitaph(self, /):
        return self.taphonomy.custom_epitaph(
            '$A(a)',
            dict(A=type(self)._ptolemaic_class__, a=self.dct),
            )

In [ ]:
out = MyClass({'a': 1, 'b': 2})[Chora()]
out

In [ ]:
out['b']

In [ ]:
out

In [ ]:
out is out

In [ ]:
out[:]

In [ ]:
out.chora.getitem(out, slice(None))

In [ ]:
out[:] is out

In [ ]:
out2 = out[Chora()]

In [ ]:
out2['a']